In [2]:
import scipy.stats as st
import numpy as np
np.random.seed(seed=0)

In [4]:
#### Statistical Power
## Benötigte Samplegröße mittels Simulation bestimmen

## Parameter setzen
effect_size     = 0.05 # prozentuale Steigerung
control_mean    = 1
treatment_mean  = control_mean * (1+effect_size)
control_sd      = 0.5

# Zu erziehlende Power
beta = 0.8    

sims = 1000
sample_size = 50

### Idee: 
# Sample size um 10 erhöhen bis benötigte Power erreicht
while 1:
    # Simulieren von Ctrl
    control_time_spent = np.random.normal(loc=control_mean, 
                                          scale=control_sd, 
                                          size=(sample_size, sims))
    # Simulieren von Trmt
    treatment_time_spent = np.random.normal(loc=treatment_mean, 
                                            scale=control_sd, 
                                            size=(sample_size, sims))

    # ##Notiz: 
    # treatment_time_spent = control_time_spent * (1+effect_size) GEHT NICHT!!
    # => nicht ähnlicher Ausgang - braucht nur ca. häfte => siehe nächste Methode

    # t-test
    t, p = st.ttest_ind(treatment_time_spent, control_time_spent)

    # Power = Prozent der Simulationen mit P-Wert kleiner als 0.05
    power = (p < 0.05).sum() / sims
    
    if power >= beta:        # Gewünschte Power erreicht?
        break
    else:
        sample_size += 10    # sonst nächste Simulation mit derzeitiger sample-size + 10
        
print("For {}% power, sample size required = {}".format(beta*100, sample_size))



For 80.0% power, sample size required = 1540


In [14]:
#### Power Berechnung für konkrete Daten 
# Control:   Historische Daten 
# Treatment: Historische Daten + Effect
# Dann mit Boostrapping samples generien & t-test machen

## benutzen, wenn reale Daten vorhanden auf den man simulieren mächte
# => schliesst die konkreten Charakteristiken der tatsächlichen mit ein
# => kleinerer Gap zwischen Theorie und Praxis

effect_size     = 0.05
sims = 1000
sample_size = 50

# Hier normalerweise die realen Daten rein: 
# => verteilung auf der man den Effekt nachher feststellen soll

# ABER WAS IST DANN MIT SIMS?
ctrl = np.random.normal(loc=control_mean, 
                        scale=control_sd, 
                        size=sample_size)
# => bei kleiner ausgangsmenge muss bootstrapping mit replace = True gemacht werden
#    ==> liefert sehr ähnliches Ergebnis


### Effekt auf die konkrete population nachstellen!!!
# (anstatt unabhänige mit denselben Parametern zu simulieren)
### !!! hier: Multiplikativ
trmt = ctrl * (1+effect_size)

while 1:
    select_index_control = np.random.choice(ctrl.shape[0], size=(sample_size,sims), replace=True)
    ctrl_bs_samples = ctrl[select_index_control]
    select_index_treatment = np.random.choice(trmt.shape[0], size=(sample_size,sims), replace=True)
    trmt_bs_samples = trmt[select_index_treatment]
    #  => jedes mal neues sample generieren!!!

    t, p = st.ttest_ind(trmt_bs_samples, ctrl_bs_samples)
    power = (p < 0.05).sum()/sims
    if power >= beta:
        break
    else:
        sample_size += 10
        print(f"{np.round(power,4)}: {sample_size}")
        
print("For 80% power, sample size required = {}".format(sample_size))

0.089: 60
0.091: 70
0.08: 80
0.086: 90
0.084: 100
0.092: 110
0.105: 120
0.119: 130
0.112: 140
0.115: 150
0.126: 160
0.106: 170
0.145: 180
0.123: 190
0.145: 200
0.166: 210
0.148: 220
0.163: 230
0.166: 240
0.162: 250
0.182: 260
0.162: 270
0.164: 280
0.208: 290
0.189: 300
0.186: 310
0.199: 320
0.189: 330
0.198: 340
0.219: 350
0.234: 360
0.242: 370
0.221: 380
0.254: 390
0.246: 400
0.236: 410
0.256: 420
0.255: 430
0.26: 440
0.27: 450
0.275: 460
0.27: 470
0.299: 480
0.291: 490
0.27: 500
0.315: 510
0.308: 520
0.317: 530
0.329: 540
0.31: 550
0.34: 560
0.319: 570
0.371: 580
0.324: 590
0.343: 600
0.332: 610
0.346: 620
0.362: 630
0.362: 640
0.359: 650
0.333: 660
0.366: 670
0.385: 680
0.397: 690
0.405: 700
0.411: 710
0.393: 720
0.381: 730
0.436: 740
0.405: 750
0.413: 760
0.449: 770
0.4: 780
0.409: 790
0.442: 800
0.427: 810
0.472: 820
0.444: 830
0.449: 840
0.447: 850
0.456: 860
0.422: 870
0.471: 880
0.502: 890
0.47: 900
0.479: 910
0.464: 920
0.518: 930
0.481: 940
0.481: 950
0.51: 960
0.514: 970
0.5

In [12]:
ctrl.shape
#select_index_control.shape

(50, 1000)

In [ ]:
### vs analytisch
from statsmodels.stats.power import TTestIndPower

effect = 0.05
alpha = 0.05
power = 0.8
sigma = .5              
cohensd = effect/sigma 

analysis = TTestIndPower()
result = analysis.solve_power(cohensd, power=power, nobs1=None, ratio=1.0, alpha=alpha)
print('Sample Size: {}'.format(np.round(result,0)))